### Modelo NLP para predecir ODS's

##### Usaremos varios modelos para ver cuál es el más efectivo

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import re
import string
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.adapt import MLkNN
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from langdetect import detect, DetectorFactory
#import es_core_web_sm

In [2]:
# Vamos a importar nuestro dataset de datos

contenidos = pd.read_excel(r"C:\Users\pueygu01\Python Programs\Máster - Proyecto\dataset - español y catalan.xlsx"
                          ,sheet_name = "Sheet1")
contenidos.head()

,title,description,Etiqueta
0,"Diferència entre publicar idea, iniciativa o e...",Idea: Una llavor de canvi. La idea pot ser qua...,ODS4
1,"Publica una idea, sembra una llavor",El detall més simple pot ser motor de canvi. D...,ODS3
2,Publica una iniciativa: ets el primer brot,Si has tingut una idea i la vols fer realitat ...,ODS11
3,Publica un esdeveniment: comenceu a recollir e...,"Tant si ets una entitat (associació, ONG, come...",ODS11
4,"Puede que cuando menos pienses, brote lo mejor...",Existe una creencia popular de que cuando las ...,ODS3 ODS16


In [3]:
# Podemos limpiar un poco el texto de mayúsculas y comillas. Para ello usamos una función que nos lo haga en todo el dataset

def clean_text(text):
    text = text.lower()
    text = text.strip(' ')
    text = text.strip('"')
    return text

In [4]:
# Aplicamos la función anterior

contenidos["catalan_limpio"]=contenidos.description.apply(clean_text)
contenidos.head()

,title,description,Etiqueta,catalan_limpio
0,"Diferència entre publicar idea, iniciativa o e...",Idea: Una llavor de canvi. La idea pot ser qua...,ODS4,idea: una llavor de canvi. la idea pot ser qua...
1,"Publica una idea, sembra una llavor",El detall més simple pot ser motor de canvi. D...,ODS3,el detall més simple pot ser motor de canvi. d...
2,Publica una iniciativa: ets el primer brot,Si has tingut una idea i la vols fer realitat ...,ODS11,si has tingut una idea i la vols fer realitat ...
3,Publica un esdeveniment: comenceu a recollir e...,"Tant si ets una entitat (associació, ONG, come...",ODS11,"tant si ets una entitat (associació, ong, come..."
4,"Puede que cuando menos pienses, brote lo mejor...",Existe una creencia popular de que cuando las ...,ODS3 ODS16,existe una creencia popular de que cuando las ...


In [5]:
# Para poder aplicar un modelo de prediccón de multi-etiqueta, es necesario que las etiquetas sean un conjunto de listas
# Pasamos a lista nuestras etiquetas

#contenidos_castellano["Etiqueta"] = contenidos_castellano["Etiqueta"].str.split()
#contenidos_catalan["Etiqueta"] = contenidos_catalan["Etiqueta"].str.split()
contenidos["Etiqueta"] = contenidos["Etiqueta"].str.split()

In [6]:
# Con este paquete podemos extraer las etiquetas y ponerlas como columnas ya que es así cómo los modelos pueden trabajr con el dataset

from sklearn.preprocessing import MultiLabelBinarizer
multilabel = MultiLabelBinarizer()

#new_contenidos_castellano = multilabel.fit_transform(contenidos_castellano["Etiqueta"])
#new_contenidos_catalan = multilabel.fit_transform(contenidos_catalan["Etiqueta"])
new_contenidos = multilabel.fit_transform(contenidos["Etiqueta"])


In [7]:
# Para ver todas las diferentes etiquetas, llamamos a "classes_" mediante miltilabel
multilabel.classes_

array(['ODS1', 'ODS10', 'ODS11', 'ODS12', 'ODS13', 'ODS14', 'ODS15',
       'ODS16', 'ODS17', 'ODS1O', 'ODS2', 'ODS3', 'ODS4', 'ODS5', 'ODS6',
       'ODS8', 'ODS9', 'OSD3'], dtype=object)

In [8]:
# Para poder trabajar, lo tenemos que pasar a DataFrame y aprovechamos para poner las etiquetas como columnas

#new_contenidos_castellano = pd.DataFrame(new_contenidos_castellano
#             , columns = multilabel.classes_)

#new_contenidos_catalan = pd.DataFrame(new_contenidos_catalan
#            , columns = multilabel.classes_)
new_contenidos1 = pd.DataFrame(new_contenidos
             , columns = multilabel.classes_)
new_contenidos1.head()

,ODS1,ODS10,ODS11,ODS12,ODS13,ODS14,ODS15,ODS16,ODS17,ODS1O,ODS2,ODS3,ODS4,ODS5,ODS6,ODS8,ODS9,OSD3
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [9]:
# Concatenamos el último dataframe con su correspondiente comentario para obtener así el formato de tabla que necesitamos
# para moderlar

#new_contenidos_castellano = pd.concat([contenidos_castellano["español"], new_contenidos_castellano], axis=1)
#new_contenidos_catalan = pd.concat([contenidos_catalan["catalan"], new_contenidos_catalan], axis=1)
df_new_contenidos = pd.concat([contenidos, new_contenidos1], axis=1)

df_new_contenidos.head()

,title,description,Etiqueta,catalan_limpio,ODS1,ODS10,ODS11,ODS12,ODS13,ODS14,...,ODS17,ODS1O,ODS2,ODS3,ODS4,ODS5,ODS6,ODS8,ODS9,OSD3
0,"Diferència entre publicar idea, iniciativa o e...",Idea: Una llavor de canvi. La idea pot ser qua...,[ODS4],idea: una llavor de canvi. la idea pot ser qua...,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,"Publica una idea, sembra una llavor",El detall més simple pot ser motor de canvi. D...,[ODS3],el detall més simple pot ser motor de canvi. d...,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Publica una iniciativa: ets el primer brot,Si has tingut una idea i la vols fer realitat ...,[ODS11],si has tingut una idea i la vols fer realitat ...,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Publica un esdeveniment: comenceu a recollir e...,"Tant si ets una entitat (associació, ONG, come...",[ODS11],"tant si ets una entitat (associació, ong, come...",0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Puede que cuando menos pienses, brote lo mejor...",Existe una creencia popular de que cuando las ...,"[ODS3, ODS16]",existe una creencia popular de que cuando las ...,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# Creación de Modelos
------------------------

### KNN - Vecinos más cercanos (`Castellano`)

In [10]:
print(df_new_contenidos.columns[5:])

Index(['ODS10', 'ODS11', 'ODS12', 'ODS13', 'ODS14', 'ODS15', 'ODS16', 'ODS17',
       'ODS1O', 'ODS2', 'ODS3', 'ODS4', 'ODS5', 'ODS6', 'ODS8', 'ODS9',
       'OSD3'],
      dtype='object')


In [11]:
# Definimos un vector X con los comentarios y un vector y con las etiquetas (1 ó 0)

X_castellano = df_new_contenidos["español_limpio"]
y_castellano = np.asarray(df_new_contenidos[df_new_contenidos.columns[5:]])

# Iniciamos el paquete TfidfVectorizer 

vetorizar_castellano = TfidfVectorizer(max_features=3000, max_df=0.85)

# Aplicamos el fit() en el vector de comentarios X
vetorizar_castellano.fit(X_castellano)

# Usando "train_test_split" partimos nuestro dataset en entrenamiento y test
X_train_castellano, X_test_castellano, y_train_castellano, y_test_castellano = train_test_split(X_castellano
                                                                                               ,y_castellano
                                                                                               ,test_size=0.25
                                                                                               ,random_state=42)

# Transformamos nuestros datos

X_castellano_train_tfidf = vetorizar_castellano.transform(X_train_castellano)
X_castellano_test_tfidf = vetorizar_castellano.transform(X_test_castellano)

# Vamos a usar Multilabel de los vecinos más cercanos

mlknn_classifier_castellano = MLkNN()
mlknn_classifier_castellano.fit(X_castellano_train_tfidf, y_train_castellano)

KeyError: 'español_limpio'

### KNN - Vecinos más cercanos (`Catalán`)

In [12]:
# Definimos un vector X con los comentarios y un vector y con las etiquetas (1 ó 0)

X_catalan = df_new_contenidos["catalan_limpio"]
y_catalan = np.asarray(df_new_contenidos[df_new_contenidos.columns[5:]])

# Iniciamos el paquete TfidfVectorizer 

vetorizar_catalan = TfidfVectorizer(max_features=3000, max_df=0.85)

# Aplicamos el fit() en el vector de comentarios X
vetorizar_catalan.fit(X_catalan)

# Usando "train_test_split" partimos nuestro dataset en entrenamiento y test
X_train_catalan, X_test_catalan, y_train_catalan, y_test_catalan = train_test_split(X_catalan
                                                                                   ,y_catalan
                                                                                   ,test_size=0.25
                                                                                   ,random_state=42)

# Transformamos nuestros datos

X_catalan_train_tfidf = vetorizar_catalan.transform(X_train_catalan)
X_catalan_test_tfidf = vetorizar_catalan.transform(X_test_catalan)

# Vamos a usar Multilabel de los vecinos más cercanos

mlknn_classifier_catalan = MLkNN()
mlknn_classifier_catalan.fit(X_catalan_train_tfidf, y_train_catalan)

C:\Users\pueygu01\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_neighbors=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


MLkNN()

### Generación de las dos funciones. Una para cada idioma

In [13]:
def identificador_ODS_castellano(comentario):
    frase_tfidf = vetorizar_castellano.transform(comentario)
    predicción = multilabel.inverse_transform(mlknn_classifier_castellano.predict(frase_tfidf))
    return print("El/los ODS para ese comentario son;", predicción)

def identificador_ODS_catalan(comentario):
    frase_tfidf = vetorizar_catalan.transform(comentario)
    predicción = multilabel.inverse_transform(mlknn_classifier_catalan.predict(frase_tfidf))
    return print("El/los ODS para ese comentario son;", predicción)

## Cargamos el dataset de la aplicació

In [14]:
txt = pd.read_excel("contenidos y etiquetas_proba.xlsx", index_col=None)
txt.head(10)

,title,description,Etiqueta
0,La SEXUALITAT AL TEU ABAST ??,Taller de Sexualitat: “El plaer masculí” amb l...,ODS5
1,"Redescobreix el nostre entorn natural amb ""La ...",? Vols reconnectar amb la natura? Estimula tot...,ODS13
2,MULLA'T PER L'ESCLEROSI MÚLTIPLE.,"Bon dia a tothom, com cada any per aquestes da...",ODS3
3,"3a Gala Lírica ""Els Claustres de Breda""",Llancem una iniciativa per ajudar al menjador ...,NaN


# Función para limpiar el texto

In [15]:
#lista de stop-words específicos de nuestro corpus (aproximación)
stop_words = ['el', 'la', 'lo', 'los', 'las', 'un', 'una', 'unos', 'unas', 'me', 'a', 'de', 'se', 'te','?']

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    eliminamos los tokens que son signos de puntuación, convertimos en
    minúsculas y quitamos signos de puntuación. Para terminar
    volvemos a convertir en cadena de texto"""
    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lower_ for tok in tokens if not tok.is_punct and not tok.is_space]
    filtered_tokens = [pattern2.sub('', token) for token in tokens if not (token in stop_words)]
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text

def lemmatize_text(text):
    """Convertimos el texto a tokens, extraemos el lema de cada token
    y volvemos a convertir en cadena de texto"""
    tokens = nlp(text)
    lemmatized_tokens = [tok.lemma_ for tok in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    
    return lemmatized_text

## Iterable por el dat set de la aplicación y cuando encuentra, en la columna del ODS que esta vacio entonces aplcamos el modelo predictivo

In [16]:
DetectorFactory.seed = 0
for t in txt.index:
    if pd.isnull(txt['Etiqueta'][t]):
        lengua=detect(txt['description'][t])
        if lengua=='ca':
            nlp = spacy.load("ca_core_news_sm")
            txt["lemas"]=txt.description.apply(clean_text)
            txt["lemas"]=txt.lemas.apply(lemmatize_text)
            identificador_ODS_catalan([txt['description'][t]])
        else:
            nlp = spacy.load("es_core_news_sm")
            txt.description.apply(clean_text)
            txt.description.apply(lemmatize_text)
            identificador_ODS_castellano([txt['description'][t]])
    

print(lengua)

ValueError: Expected indicator for 18 classes, but got 17

## Rendimiento de lo modelos

In [140]:
print(accuracy_score(y_test, predicted))
print(hamming_loss(y_test, predicted))

NameError: name 'predicted' is not defined

In [18]:
prediction = mlknn_classifier_catalan.predict(X_catalan_test_tfidf)

In [19]:
print(accuracy_score(y_test_catalan,prediction))

0.496


In [23]:
y_test_catalan

AttributeError: 'numpy.ndarray' object has no attribute 'describe'

### Relevancia binaria


In [25]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

In [26]:
bin_class_catalan = BinaryRelevance(GaussianNB())
bin_class_catalan.fit(X_catalan_train_tfidf, y_train_catalan)

BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [27]:
prediccion_catalan = bin_class_catalan.predict(X_catalan_test_tfidf)
print(accuracy_score(prediccion_catalan, y_test_catalan))

0.44


In [18]:
from sklearn.linear_model import LogisticRegression
from skmultilearn.problem_transform import LabelPowerset
label_powerset_catalan = LabelPowerset(LogisticRegression())
# Training logistic regression model on train data
label_powerset_catalan.fit(X_catalan_train_tfidf, y_train_catalan)

prediccion_catalan = label_powerset_catalan.predict(X_catalan_test_tfidf)
print(accuracy_score(prediccion_catalan, y_test_catalan))

0.52
